<a href="https://colab.research.google.com/github/codedyasai/Python_MachineLearning/blob/main/10_%ED%99%95%EC%9D%B8%ED%95%99%EC%8A%B5_%EC%83%9D%EC%A1%B4%EC%9E%90_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문제 정의

- 로지스틱 회귀(Logistic Regression)을 이용한 타이타닉 생존자 예측
- 목표: 타이타닉 승객 데이터셋을 이용하여 생존 여부 예측

## Library Loading


In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install mglearn
import mglearn

# 음수표현 라이브러리
plt.rcParams['axes.unicode_minus'] = False

# 경고무시
import warnings
warnings.filterwarnings("ignore")

# 매직명령어 : 시각화 결과가 노트북에 포함되도록
%matplotlib inline

## Data Loading

In [ ]:
df = pd.read_csv('titanic.csv')
df.head(5)

,Unnamed: 0,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Embarked,Survived
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,S,0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,C,1
2,2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,S,1
3,3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,S,1
4,4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,S,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  889 non-null    int64  
 1   Pclass      889 non-null    int64  
 2   Name        889 non-null    object 
 3   Sex         889 non-null    object 
 4   Age         889 non-null    float64
 5   SibSp       889 non-null    int64  
 6   Parch       889 non-null    int64  
 7   Ticket      889 non-null    object 
 8   Embarked    889 non-null    object 
 9   Survived    889 non-null    int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 69.6+ KB


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,889.0,444.000000,256.776492,0.00,222.0,444.0,666.0,888.0
Pclass,889.0,2.311586,0.834700,1.00,2.0,3.0,3.0,3.0
Age,889.0,29.315152,12.984932,0.42,22.0,28.0,35.0,80.0
SibSp,889.0,0.524184,1.103705,0.00,0.0,0.0,1.0,8.0
Parch,889.0,0.382452,0.806761,0.00,0.0,0.0,0.0,6.0
Survived,889.0,0.382452,0.486260,0.00,0.0,0.0,1.0,1.0


In [ ]:
df.corr()

,Unnamed: 0,Pclass,Age,SibSp,Parch,Survived
Unnamed: 0,1.000000,-0.035345,0.031321,-0.057692,-0.001663,-0.004997
Pclass,-0.035345,1.000000,-0.336512,0.081656,0.016824,-0.335549
Age,0.031321,-0.336512,1.000000,-0.232543,-0.171485,-0.069822
SibSp,-0.057692,0.081656,-0.232543,1.000000,0.414542,-0.034040
Parch,-0.001663,0.016824,-0.171485,0.414542,1.000000,0.083151
Survived,-0.004997,-0.335549,-0.069822,-0.034040,0.083151,1.000000


In [ ]:
df.Survived.value_counts()

0    549
1    340
Name: Survived, dtype: int64

In [ ]:
data = df[['Pclass', 'Age', 'SibSp', 'Parch']].to_numpy()
target = df['Survived'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, stratify= target, random_state= 42)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV


In [ ]:
# 개별 모델 객체 생성
# L1(Lasso), L2(Ridge)
lr = LogisticRegression(solver= 'liblinear')
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier(n_jobs=-1)

# 소프트 보팅으로 구현
vo = VotingClassifier(estimators= [('LR', lr), ('KNN', knn),('DT', dt) , ['RF', rf]], voting= 'soft')

# 학습
vo.fit(train_input, train_target)

# 예측
pred = vo.predict(test_input)

from sklearn.metrics import accuracy_score
# 정확도
print('Voting 분류기 정확도: ', accuracy_score(test_target, pred))

# 모델별 학습/예측/평가
models = [lr, knn, dt, rf]
for model in models:
    model.fit(train_input, train_target)
    pred = model.predict(test_input)
    model_name = model.__class__.__name__
    score = accuracy_score(test_target, pred)
    print(f'{model_name} 정확도: {score:.4f}')

Voting 분류기 정확도:  0.695067264573991
LogisticRegression 정확도: 0.6996
KNeighborsClassifier 정확도: 0.7085
DecisionTreeClassifier 정확도: 0.7130
RandomForestClassifier 정확도: 0.6951


- RandomForest

- LogisticRegression